<a href="https://colab.research.google.com/github/kaikoba/Skaters/blob/master/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
from pandas import DataFrame, Series
import category_encoders as ce
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [ ]:
df_train = pd.read_csv('/content/sample_data/train.csv', index_col=0)
df_test = pd.read_csv('/content/sample_data/test.csv', index_col=0)

In [ ]:
y_train = df_train.y
x_train = df_train.drop(['y'], axis=1)
x_test = df_test

In [ ]:
cats = []
for col in x_train.columns:
  if x_train[col].dtype == 'object':
    cats.append(col)

oe = ce.OrdinalEncoder(cols=cats, return_df=False)
x_train[cats] = oe.fit_transform(x_train[cats])
x_test[cats] = oe.transform(x_test[cats])

In [ ]:
x_train=x_train.fillna(x_train.median())
x_test=x_test.fillna(x_test.median())

In [ ]:
xgb_model = xgb.XGBRegressor()
reg_xgb = GridSearchCV(xgb_model,
                    {'max_depth': [2,4,6],
                     'n_estimators': [50,100,200]}, verbose=1)
reg_xgb.fit(x_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[04:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[04:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:35:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:35:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:35:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:35:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:35:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:35:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:35:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  5.4min finished


[04:40:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=None, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [2, 4, 6],
                         'n_estimators': [50, 100, 200]},
             pre_dispa

In [ ]:
pred = pd.DataFrame( {'XGB': reg_xgb.predict(x_test)})
#小数を丸めている
prediction_XG = np.round(pred)

In [ ]:
#小数を丸めている
prediction_XG

,XGB
0,406.0
1,141.0
2,96.0
3,258.0
4,159.0
...,...
18523,170.0
18524,159.0
18525,281.0
18526,137.0


In [ ]:
# ----- Submit dataの作成　------- 
submissionfile = pd.read_csv('/content/sample_data/sample_submit.csv', index_col=0)
submissionfile['y'] = prediction_XG
submissionfile.to_csv('./submissionfile.csv')